# World Press Freedom 

## Данные 

Данные взяты с сайта: https://rsf.org/en

Ежегодно организация "Репортеры без границ" публикует свой рейтинг стран, характеризующий "доброжелательность" страны к журналистам. Среди журналистов и экспертов проводится опрос, согласно которому рассчитываются такие показатели как:плюрализм, независимость СМИ, окружение и самоцензура, законодательство, прозрачность, инфраструктура, репрессии. Далее согласно методологии рассчитывается индекс свободы прессы. Чем меньше данные показатель, тем обстановка в стране для журналистов считается благополучнее.

## Библиотеки

In [9]:
import sys

from PyQt5.QtWidgets import QMainWindow, QApplication, QPushButton, QWidget,\
    QTabWidget, QGridLayout, QTableWidget, QTableWidgetItem, QHeaderView
from PyQt5 import QtCore
import requests
import io
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import colorbar, colors
from matplotlib.backends.backend_qt4agg import FigureCanvasQTAgg as FigureCanvas
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import numpy as np

## Глобальные константы

DICT_MATCHING - сопоставление стран и их кодов (если код в исходных данных не совпадает с кодом для cartopy); FIELDS_TO_VIEW - сопоставление атрибутов полученных данных с атрибутами на просмотр.

In [10]:
URL = 'https://rsf.org/sites/default/files/index_2019_-_pour_import_1_1.csv'
DICT_MATCHING = {
    'Norway': 'NOR',
    'France': 'FRA',
    'Somaliland': 'SOM',
    'N. Cyprus': 'CTU',
    'Kosovo': 'XKO',
    'W. Sahara': 'MAR'
}
FIELDS_TO_VIEW = {
    'EN_country': 'Country',
    'Score 2019': 'Score 2019',
    'Score 2018': 'Score 2018',
    'Progression RANK': 'Progression rank',
    'Rank2019': 'Rank 2019',
    'Rank 2018': 'Rank 2018',
}

## Класс окна

In [11]:
class MainWindow(QMainWindow):

    def __init__(self):
        super().__init__()
        self.title = 'RSF'
        self.setFixedSize(1050, 700)
        self.setWindowTitle(self.title)

        self.table_widget = TabWidget(self)
        self.setCentralWidget(self.table_widget)

        self.show()

## Класс виджетов

In [12]:
class TabWidget(QWidget):

    def __init__(self, parent):
        super(QWidget, self).__init__(parent)

        self.layout = QGridLayout(self)
        self.setLayout(self.layout)

        self.df = get_data()

        # START TABS
        self.tabs = QTabWidget()

        self.tab_table = QWidget()
        self.tab_map = QWidget()

        self.tabs.addTab(self.tab_table, "Данные")
        self.tabs.addTab(self.tab_map, "Карта")
        self.layout.addWidget(self.tabs)
        # END TABS

        # START TABLE
        self.table = QTableWidget(self)

        self.table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.table.verticalHeader().setVisible(False)

        self.table.setEditTriggers(QTableWidget.NoEditTriggers)
        # END TABLE

        # START PLOTS
        self.plot_map = MatPlot(self.tab_map)
        # END PLOTS

        # START BUTTONS
        self.btn_2019 = QPushButton('2019', self.tab_map)
        self.btn_2019.setCheckable(True)
        self.btn_2019.setChecked(True)
        self.btn_2019.clicked[bool].connect(self.change_year)
        self.btn_2018 = QPushButton('2018', self.tab_map)
        self.btn_2018.clicked[bool].connect(self.change_year)
        self.btn_2018.setCheckable(True)
        # END BUTTONS

        # START FILLING TABS
        self.layout_tab_table = QGridLayout()
        self.tab_table.setLayout(self.layout_tab_table)

        self.layout_tab_map = QGridLayout()
        self.tab_map.setLayout(self.layout_tab_map)
        # END FILLING TABS

        self.set_position()

        self.fill()
        self.refresh_plot('Score 2019')

    def set_position(self):
        self.layout_tab_table.addWidget(self.table, 0, 0)

        self.layout_tab_map.addWidget(self.btn_2018, 0, 4, 1, 1)
        self.layout_tab_map.addWidget(self.btn_2019, 0, 5, 1, 1)
        self.layout_tab_map.addWidget(self.plot_map.canvas, 1, 0, 1, 10)

    def fill(self, df=None):
        if df is None:
            df = self.df[list(FIELDS_TO_VIEW.keys())]
        headers = df.columns.values.tolist()
        headers = [FIELDS_TO_VIEW[name_field] for name_field in headers if name_field in FIELDS_TO_VIEW]
        self.table.setColumnCount(len(headers))
        self.table.setHorizontalHeaderLabels(headers)
        self.table.setRowCount(df.shape[0])
        for i, row in df.iterrows():
            for j in range(self.table.columnCount()):
                self.table.setItem(i, j, QTableWidgetItem(str(row[j])))

    def refresh_plot(self, field):
        self.plot_map.get_plot(self.df, field)

    def change_year(self):
        source = self.sender()
        if source == self.btn_2019:
            if self.btn_2019.isChecked():
                self.btn_2018.setChecked(False)
                self.refresh_plot('Score 2019')
            else:
                self.btn_2019.setChecked(True)
        elif source == self.btn_2018:
            if self.btn_2018.isChecked():
                self.btn_2019.setChecked(False)
                self.refresh_plot('Score 2018')
            else:
                self.btn_2018.setChecked(True)

## Класс отрисовки карты

In [13]:
class MatPlot(object):
    def __init__(self, parent):
        shapename = 'admin_0_countries'
        self.curr_score = None
        self.countries_shp = shpreader.natural_earth(
            resolution='110m',
            category='cultural',
            name=shapename
        )

        self.fig, self.ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
        self.scale = self.fig.add_axes([0.25, 0.24, 0.5, 0.03])
        self.canvas = FigureCanvas(self.fig)
        self.canvas.setParent(parent)

    def get_plot(self, df, field='Score 2019', colormap='afmhot_r', plotscale=True, count_ticks=10):
        if self.curr_score == field:
            return
        self.ax.clear()
        self.scale.clear()

        k1 = 0.8
        k2 = 1 - k1

        cmap = plt.get_cmap(colormap)
        max_val = df[field].max()
        min_val = df[field].min()
        for country in shpreader.Reader(self.countries_shp).records():
            name_country = DICT_MATCHING.get(country.attributes['NAME'], country.attributes['ISO_A3'])
            if name_country in df['ISO'].values:
                val = df[df['ISO'] == name_country][field].values[0]
                color = cmap([val / max_val * k1 + k2]).ravel().tolist()
            else:
                color = '#ffffff'

            self.ax.add_geometries(
                country.geometry,
                ccrs.PlateCarree(),
                facecolor=color
            )

        self.ax.set_title('World Press Freedom', size=12)
        if plotscale:
            min_color = round(min_val / max_val * k1 + k2, 2)
            cbar = colorbar.ColorbarBase(
                self.scale,
                cmap=cmap,
                norm=colors.Normalize(vmin=min_color, vmax=1),
                orientation='horizontal',
                ticks=np.round(np.linspace(min_color, 1, count_ticks), 2)
            )
            cbar.set_clim(0, 1)
            cbar.ax.set_title(field)
            cbar.ax.set_xticklabels(np.round(np.linspace(min_val, max_val, count_ticks), 2))

        self.curr_score = field
        self.canvas.draw()

## Класс данных

In [14]:
def get_data():
    response = requests.get(URL)

    with io.StringIO(response.content.decode('utf-8')) as f:
        return pd.read_csv(f, decimal=',')

In [15]:
app = QtCore.QCoreApplication.instance()
if app is None:
    app = QtWidgets.QApplication(sys.argv)
ex = MainWindow()
app.exec_()

-1